In [ ]:
import json
import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import HTML
import frontmatter

In [ ]:
df = pd.read_pickle('tidy_events.pkl.gz')

In [ ]:
def df_to_json(df:pd.DataFrame, fn:str):
    """将带有日期的df转成多级json
    df.index: a str format data, eg '1969-04-21'
    to_dict make this as key.
    then break the key into year and month and put dates into
    """
    # 原始字典
    original_dict = df.to_dict('index')

    # 转换后的字典
    nested_dict = {}

    for date_key, inner_dict in original_dict.items():
        year, month, day = date_key.split('-')
        
        if year not in nested_dict:
            nested_dict[year] = {}
        
        if month not in nested_dict[year]:
            nested_dict[year][month] = {}
        
        nested_dict[year][month][date_key] = inner_dict

    # 将多级字典存储为 JSON 文件
    with open(fn, 'w') as json_file:
        json.dump(nested_dict, json_file, indent=4)

    print(f"多级字典已成功存储为{fn}文件")

In [ ]:
df_to_json(df, 'events.json')

In [ ]:
def add_fm(folder):
    meta = {
    'layout':'../../layouts/Layout.astro'
}
    folder = Path(folder)
    fs = folder.glob('*.md')
    for f in fs:
        post = frontmatter.load(f)
        post.metadata = meta
        frontmatter.dump(post, f)
    print('done!')


In [ ]:
def create_urls():
    htmls_d = Path('src/components/htmls')
    url_d = Path('src/pages/reports')
    url_d.mkdir(exist_ok=True)

    def make_report(out_d, stem):
        out_fn = out_d / f"{stem}.astro"  # 使用 f-string 简化文件名拼接

        report = f"""---
    import Report from '@/components/htmls/{stem}.html'
    import Layout from '@/layouts/Layout.astro'
    ---
    <Layout title='{stem}'>
        <Report />
    </Layout>

    """
        # print(f'report: {report}')  # 打印替换后的内容

        # 将修改后的内容写入新文件
        with open(out_fn, 'w', encoding='utf-8') as file:
            file.write(report)

    fs = htmls_d.glob('*.html')
    for f in fs:
        stem = str(f.stem)
        print(f'url:{stem}')
        make_report(url_d, stem)


In [ ]:
lable_css = """
<style>
    .mylab-group {
        display: flex;
        flex-flow: row wrap;
    }
    .mylab {
        display:flex;
        justify-content: center;
        border: 1px solid #ccc;
        border-radius: 5px;
        padding: 2px;
        margin: 2px;
        width: 5em;
        background-color: #f9f9f9;
    }
    .mylab p {
        color: #666;
        font-size: 0.8em;
        margin: 0;
    }
</style>
"""

def s_to_lable(s, title=None):
    """
    s可以为列表或者pd.Series
    """
    def create_lable(title):
        
        card_html = f"""
        <div class='mylab'>
            <p>{title}</p>
        </div>
        """
        return card_html

    def create_lable_group(cards_html):
        card_group_html = f"""
        <div class='mylab-group'>
            {cards_html}
        </div>
        """
        return card_group_html
    title_html = f'<h2>{title}</h2>\n' if title else ""
    labs = [create_lable(i) for i in s]
    labs_html = ''.join(labs)
    group_html = create_lable_group(labs_html) + lable_css
    display(HTML(title_html+group_html))
    # return group_html

In [ ]:
# test = pd.Series(range(10), name='num')
test = pd.Series(['中国银行']*10, name='num')
s_to_lable(test, 'this is title')

In [ ]:
s_to_lable(['中国银行']*10)